In [1]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [2]:
def driver_setup():# Assigning a Headless Firefox Driver
  options = webdriver.FirefoxOptions()
  options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
  options.add_argument('E:/Programs/geckodriver.exe')
  # options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [3]:
i = 0
driver = driver_setup()

The version of firefox cannot be detected. Trying with latest driver version


In [4]:
url = 'https://www.naukri.com/jobs-in-india-1?clusters=functionalAreaGid&functionAreaIdGid=1&functionAreaIdGid=3&functionAreaIdGid=4&functionAreaIdGid=5&functionAreaIdGid=7&functionAreaIdGid=8&functionAreaIdGid=9&functionAreaIdGid=11&functionAreaIdGid=14&functionAreaIdGid=15&functionAreaIdGid=19&functionAreaIdGid=25&cityTypeGid=4&cityTypeGid=6&cityTypeGid=17&cityTypeGid=51&cityTypeGid=64&cityTypeGid=65&cityTypeGid=73&cityTypeGid=97&cityTypeGid=110&cityTypeGid=125&cityTypeGid=134&cityTypeGid=135&cityTypeGid=138&cityTypeGid=139&cityTypeGid=173&cityTypeGid=183&cityTypeGid=184&cityTypeGid=220&cityTypeGid=232&cityTypeGid=323&cityTypeGid=9508&cityTypeGid=9509&jobAge=1'
# url = 'https://www.naukri.com/energy-jobs?k=energy&nignbevent_src=jobsearchDeskGNB&functionAreaIdGid=3&functionAreaIdGid=19'
driver.get(url)

In [17]:
# i = 0
save = 1
while True:
    
    time.sleep(4)
    
    # job_articles = driver.find_elements(By.CLASS_NAME, 'jobTuple')
    job_articles = driver.find_elements(By.CLASS_NAME, 'cust-job-tuple')
    for job_element in job_articles:
        # Find the title element within the job element
        
        try:
            job_title = job_element.find_element(By.CLASS_NAME, 'title').text
        except:
            job_title = "N/A"
        
        try:
            company_name = job_element.find_element(By.CLASS_NAME, 'comp-name').text
        except:
            company_name = "N/A"
        
        try:
            experience = job_element.find_element(By.CLASS_NAME, 'expwdth').text
        except:
            experience = "N/A"
        
        try:
            salary = job_element.find_element(By.CLASS_NAME, 'sal').text
            if (salary == 'Not disclosed')or(salary == 'Unpaid'):
                continue
        except:
            continue
        
        try:
            location = job_element.find_element(By.CLASS_NAME, 'locWdth').text
        except:
            location = "N/A"
        
        try:
            description = job_element.find_element(By.CLASS_NAME, 'job-desc').text
        except:
            description = "N/A"
        
        try:
            li_elements = job_element.find_elements(By.CSS_SELECTOR, 'ul.tags-gt li.tag-li')
            l = [i.text for i in li_elements]
        except:
            l = []
        
        try:
            posted_on = job_element.find_element(By.CLASS_NAME, 'job-post-day').text
        except:
            days_ago = "N/A"
        
        current_date = datetime.datetime.now().date()
        data = {
            'job_title':job_title,
            'company_name':company_name,
            'experience':experience,
            'salary':salary,
            'location':location,
            'description':description,
            'posted_on':posted_on,
            'current_date':current_date
        }
        
        for i in range(len(l)):
            data['skill'+str(i+1)]=l[i]
            
        try:
            job_element.find_element(By.TAG_NAME, 'a').click()
            time.sleep(3)
            driver.switch_to.window(driver.window_handles[1])
            poi = driver.find_element(By.CLASS_NAME, 'styles_other-details__oEN4O')
            div_elements = poi.find_elements(By.CLASS_NAME, 'styles_details__Y424J')

            for eleind in range(len(div_elements)):
                label = div_elements[eleind].find_element(By.XPATH, './/label').text.strip(':').lower()
                ans = div_elements[eleind].find_element(By.XPATH, './/label/following-sibling::span').text.lower()
                data[label] = ans
            
            data['job_description'] = driver.find_element(By.CLASS_NAME, "styles_JDC__dang-inner-html__h0K4t").text

            driver.close()
            driver.switch_to.window(driver.window_handles[0])        
        except Exception as e:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue
        
        if data != {}:
            print(f'\rProgress: {save}', end='')
            save+=1
            tempdf = pd.DataFrame(data, index=[0])
            tempfilename = 'Naukri_raw.csv'
            # tempfilename = 'energy_roles.csv'#---------------------------------------------------------------------------------
            save_data_to_disk(tempdf, tempfilename)
            tempdf = pd.DataFrame()
    
    # next_button = driver.find_elements(By.CLASS_NAME, 'styles_btn-secondary__2AsIP')[1]
    # next_button = driver.find_element(By.CSS_SELECTOR, 'a.fright.fs14.btn-secondary.br2')
    time.sleep(1)
    next_button = driver.find_element(By.LINK_TEXT, "Next")
    is_disabled = next_button.get_attribute('disabled') is not None
    if is_disabled:
        break
    else:
        next_button.click()
    # i += 1

Progress: 225

NoSuchElementException: Message: Unable to locate element: Next; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:395:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


In [6]:
# ndf = pd.DataFrame()
# ndf = pd.read_excel('Naukri_Extract_20231121.xlsx')
# ndf1 = pd.read_csv(r'Naukri_raw.csv')

# ndf = pd.concat([ndf, ndf1], ignore_index=True)
# subsetcols = ndf.columns.to_list()
# subsetcols.remove('current_date')
# subsetcols.remove('posted_on')
# ndf = ndf[ndf['salary'] != 'Unpaid']

# ndf.drop_duplicates(subset=subsetcols, inplace=True)
# ndf.reset_index(drop=True,inplace=True)
# ndf.shape
# ndf.to_csv('Naukri_raw.csv',index=False)

In [7]:
# df= pd.read_csv('energy_roles.csv')
# df.to_excel('Energy_roles_Naukri.xlsx',index=False)

In [8]:
# df = pd.read_csv(r'C:\Users\PM\Desktop\jobs\Naukri_raw.csv')
# df.fillna('Nan', inplace=True)
# df.to_csv('Naukri_raw.csv',index=False)

In [18]:
df = pd.read_csv(r'Naukri_raw.csv')
subsetcols = df.columns.to_list()
subsetcols.remove('current_date')
subsetcols.remove('posted_on')
df = df[df['salary'] != 'Unpaid']

df.drop_duplicates(subset=subsetcols, inplace=True)
df.reset_index(drop=True,inplace=True)
# df.to_csv('Naukri_raw.csv',index=False)

In [10]:
# df = pd.read_csv(r'energy_roles.csv')
# subsetcols = df.columns.to_list()
# subsetcols.remove('current_date')
# subsetcols.remove('posted_on')
# df = df[df['salary'] != 'Unpaid']

# df.drop_duplicates(subset=subsetcols, inplace=True)
# df.reset_index(drop=True,inplace=True)

# current_time = datetime.datetime.now()
# filename = 'Energy_Extract_'+ str(current_time.date()).replace('-', '')+'.xlsx'
# df.to_excel(filename,index = False)

In [19]:
df.shape

(61423, 22)

In [12]:
# driver.quit()

In [20]:
df.columns

Index(['job_title', 'company_name', 'experience', 'salary', 'location',
       'description', 'posted_on', 'current_date', 'skill1', 'skill2',
       'skill3', 'skill4', 'skill5', 'skill6', 'skill7', 'skill8', 'role',
       'industry type', 'department', 'employment type', 'role category',
       'job_description'],
      dtype='object')

In [21]:
def clean_text(text):
    if pd.isna(text):
        return ""
    
    # Replace specific characters that cause issues in Excel
    cleaned_text = re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F]+', '', str(text))
    return cleaned_text

# Apply the clean_text function to the relevant columns in your DataFrame
df['location'] = df['location'].apply(clean_text)
df['job_description'] = df['job_description'].apply(clean_text)

In [22]:
current_time = datetime.datetime.now()
filename = 'Naukri_Extract_'+ str(current_time.date()).replace('-', '')+'.xlsx'
df.to_excel(filename,index = False)

In [23]:
'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21205009&cid=0F87FC3AA6EB46E0'

'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21205009&cid=0F87FC3AA6EB46E0'